In [47]:
import numpy as np
import pandas as pd
import xml.dom.minidom as xdm
#打开xml文档


In [2]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [54]:
from keras import train_test

In [78]:
def dist(a,b):
    return np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2 +(a[2]-b[2])**2)

def get_data(atomsets):
    X=[];Y=[]
    print(len(atomsets))
    num=1
    for atomset in atomsets:
        print(num)
        num+=1
        unit_cell=atomset.getElementsByTagName('unit_cell')
        d=float(unit_cell[0].getAttribute('a').split()[0])

        poses=atomset.getElementsByTagName('position')
        forces=atomset.getElementsByTagName('force')
        # put back  the outside nodes
        N=280
        assert len(poses)==len(forces)==N
        H_pos=[];c_pos=[]
        pos_all=[]
        force_all=[]

        for i in range(N):
            p=poses[i].firstChild.data.split()
            p=np.array(list(map(float,p)))
            p=np.mod(p,d)
            pos_all.append(p)
            force_all.append(forces[i].firstChild.data.split())
        pos_mat=[]
        p=np.array(pos_all)
        f=np.array(force_all).astype(float)

        for i in range(N):
            pp=p-p[i]
            o=np.where(pp>d/2)
            pp[o]=d-pp[o]
            R2=np.sum(np.power(pp,2),axis=1)
            R2=np.delete(R2,i,axis=0)
            pp=np.delete(pp,i,axis=0)
            hi=np.argsort(R2[:224])
            ci=np.argsort(R2[224:])
            ri=np.append(hi,ci)
            R2=R2[ri]
            pp=pp[ri]
            hf=np.array(pp[:64]/(R2[:64].reshape(64,1)))
            hf=np.concatenate(((1/np.sqrt(R2[:64])).reshape(64,1),hf),axis=1)
            hf=hf.flatten()
            cf=np.array(pp[224:240]/(R2[224:240].reshape(16,1)))
            cf=np.concatenate(((1/np.sqrt(R2[224:240])).reshape(16,1),cf),axis=1)
            cf=cf.flatten()
            pos_mat.append(np.append(np.append(hf,1/np.sqrt(R2[64:224])),np.append(cf,1/np.sqrt(R2[240:]))))
        assert len(pos_mat)==280 and len(pos_mat[0])==519
        X=X+pos_mat
        Y=Y+f.tolist()
        return X,Y

In [61]:
# X=[];Y=[]
# for i in range(1,6):
#     dom = xdm.parse('CH4-traj/%s/resultout'%(i))
#     atomsets=dom.getElementsByTagName('atomset')
#     x,y=get_data(atomsets)
#     X+=x
#     Y+=Y
#     print(i,'finished.')
# np.save('X',X)
# np.save('Y',Y)

KeyboardInterrupt: 

In [121]:
X=[];Y=[]

dom = xdm.parse('CH4-traj/1/example1')
atomsets=dom.getElementsByTagName('atomset')
x,y=get_data(atomsets)
X+=x
Y+=y
np.save('X',X)
np.save('Y',Y)

2
1


In [80]:
X

[array([ 4.98253293e-01,  3.49538791e-01, -1.02925452e-01,  3.39831325e-01,
         3.06808492e-01, -8.03900971e-02,  4.59291322e-02,  2.92505380e-01,
         3.04210963e-01,  2.74447263e-01,  8.26012856e-02,  1.01980575e-01,
         2.72797860e-01,  1.81469530e-01, -1.89910082e-01,  7.36318075e-02,
         2.29482069e-01,  1.43874847e-01, -3.29575763e-02, -1.75715243e-01,
         2.29391877e-01, -7.79051052e-03, -1.39830122e-01, -1.81679603e-01,
         2.07089239e-01,  1.20795685e-01, -3.55308095e-02, -1.64413858e-01,
         2.04428656e-01,  3.26792909e-02, -1.61950338e-01,  1.20396127e-01,
         1.92724698e-01,  1.45488945e-01, -1.22501427e-01,  3.11315938e-02,
         1.77616680e-01,  8.46823157e-02, -1.51727557e-01, -3.68149298e-02,
         1.70955297e-01,  1.62808327e-01,  1.43118070e-02, -5.01431382e-02,
         1.63095087e-01,  7.38259211e-02,  1.41939492e-02,  1.44735181e-01,
         1.53984397e-01,  3.17619445e-02, -6.34614192e-02,  1.36656583e-01,
         1.5

In [4]:
a=(pos[-1].firstChild.data.split())
a=list(map(float,a))
print(a)

[-4.94112963, 20.97056117, 8.98412066]


In [114]:
x=[];y=[]
for i in range(len(pos)):
    x.append(list(map(float,pos[i].firstChild.data.split())))
    y.append(list(map(float,force[i].firstChild.data.split())))
x=np.array(x)
y=np.array(y)

TypeError: object of type 'Element' has no len()

In [116]:
def train_test_split(x,y,t_p):
    # input must be np.array
    l=len(x)
    assert(l==len(y))
    idx=np.arange(l)
    np.random.shuffle(idx)
    train_idx=idx[:int(len(x)*t_p)]
    test_idx=idx[int(len(x)*t_p):]
    return x[train_idx],y[train_idx],x[test_idx],y[test_idx]
    

In [121]:
train_x,train_y,test_x,test_y=train_test_split(x,y,0.8)

In [134]:
XX=tf.keras.utils.normalize(
    X, axis=-1, order=1
)

In [133]:
lll=[[100,10],[200,5]]
tf.keras.utils.normalize(
    lll, axis=-1, order=1
)

array([[0.90909091, 0.09090909],
       [0.97560976, 0.02439024]])

In [135]:
sum(XX[0])

0.7148997428261835

In [167]:
model=tf.keras.Sequential([
    tf.keras.layers.Dense(64,activation=tf.nn.relu,input_shape=(519,)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(3)
])
model.compile(loss='mse',
                optimizer='Adam',
                metrics=['accuracy']) 

In [168]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 64)                33280     
_________________________________________________________________
dense_36 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_37 (Dense)             (None, 3)                 387       
Total params: 41,987
Trainable params: 41,987
Non-trainable params: 0
_________________________________________________________________


In [169]:
model.fit(np.array(XX),np.array(Y),epochs=300,verbose=1,
         validation_split=0.2)

Train on 224 samples, validate on 56 samples
Epoch 1/300
224/224 [==============================] - 0s 2ms/sample - loss: 7.0751e-04 - accuracy: 0.3661 - val_loss: 0.0027 - val_accuracy: 0.4286
Epoch 2/300
224/224 [==============================] - 0s 185us/sample - loss: 7.0605e-04 - accuracy: 0.3348 - val_loss: 0.0027 - val_accuracy: 0.2857
Epoch 3/300
224/224 [==============================] - 0s 298us/sample - loss: 6.9953e-04 - accuracy: 0.3750 - val_loss: 0.0027 - val_accuracy: 0.2500
Epoch 4/300
224/224 [==============================] - 0s 153us/sample - loss: 6.9008e-04 - accuracy: 0.3616 - val_loss: 0.0027 - val_accuracy: 0.3929
Epoch 5/300
224/224 [==============================] - 0s 139us/sample - loss: 6.7935e-04 - accuracy: 0.4107 - val_loss: 0.0027 - val_accuracy: 0.2679
Epoch 6/300
224/224 [==============================] - 0s 194us/sample - loss: 6.7380e-04 - accuracy: 0.4420 - val_loss: 0.0027 - val_accuracy: 0.2500
Epoch 7/300
224/224 [==============================

Epoch 55/300
224/224 [==============================] - 0s 163us/sample - loss: 1.5502e-04 - accuracy: 0.7902 - val_loss: 0.0029 - val_accuracy: 0.3929
Epoch 56/300
224/224 [==============================] - 0s 178us/sample - loss: 1.4513e-04 - accuracy: 0.7946 - val_loss: 0.0029 - val_accuracy: 0.3750
Epoch 57/300
224/224 [==============================] - 0s 147us/sample - loss: 1.3390e-04 - accuracy: 0.8214 - val_loss: 0.0029 - val_accuracy: 0.4107
Epoch 58/300
224/224 [==============================] - 0s 141us/sample - loss: 1.3491e-04 - accuracy: 0.7902 - val_loss: 0.0029 - val_accuracy: 0.3929
Epoch 59/300
224/224 [==============================] - 0s 127us/sample - loss: 1.2959e-04 - accuracy: 0.7991 - val_loss: 0.0029 - val_accuracy: 0.3929
Epoch 60/300
224/224 [==============================] - 0s 160us/sample - loss: 1.2475e-04 - accuracy: 0.7679 - val_loss: 0.0029 - val_accuracy: 0.3571
Epoch 61/300
224/224 [==============================] - 0s 138us/sample - loss: 1.1955e-

Epoch 109/300
224/224 [==============================] - 0s 171us/sample - loss: 1.1654e-05 - accuracy: 0.9330 - val_loss: 0.0033 - val_accuracy: 0.3571
Epoch 110/300
224/224 [==============================] - 0s 257us/sample - loss: 9.8709e-06 - accuracy: 0.9509 - val_loss: 0.0033 - val_accuracy: 0.3750
Epoch 111/300
224/224 [==============================] - 0s 133us/sample - loss: 8.6412e-06 - accuracy: 0.9509 - val_loss: 0.0033 - val_accuracy: 0.3571
Epoch 112/300
224/224 [==============================] - 0s 165us/sample - loss: 7.8206e-06 - accuracy: 0.9554 - val_loss: 0.0034 - val_accuracy: 0.3571
Epoch 113/300
224/224 [==============================] - 0s 193us/sample - loss: 8.4071e-06 - accuracy: 0.9420 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 114/300
224/224 [==============================] - 0s 158us/sample - loss: 8.2475e-06 - accuracy: 0.9375 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 115/300
224/224 [==============================] - 0s 180us/sample - loss: 7

224/224 [==============================] - 0s 183us/sample - loss: 2.6801e-07 - accuracy: 0.9911 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 163/300
224/224 [==============================] - 0s 166us/sample - loss: 2.7369e-07 - accuracy: 0.9911 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 164/300
224/224 [==============================] - 0s 181us/sample - loss: 3.2046e-07 - accuracy: 0.9955 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 165/300
224/224 [==============================] - 0s 181us/sample - loss: 3.2009e-07 - accuracy: 0.9955 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 166/300
224/224 [==============================] - 0s 222us/sample - loss: 2.8679e-07 - accuracy: 0.9911 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 167/300
224/224 [==============================] - 0s 169us/sample - loss: 2.2546e-07 - accuracy: 0.9911 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 168/300
224/224 [==============================] - 0s 191us/sample - loss: 2.1791e-07 - ac

224/224 [==============================] - 0s 186us/sample - loss: 7.3113e-09 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 216/300
224/224 [==============================] - 0s 141us/sample - loss: 7.1883e-09 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 217/300
224/224 [==============================] - 0s 140us/sample - loss: 5.9165e-09 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 218/300
224/224 [==============================] - 0s 151us/sample - loss: 4.1590e-09 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 219/300
224/224 [==============================] - 0s 154us/sample - loss: 4.1394e-09 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 220/300
224/224 [==============================] - 0s 126us/sample - loss: 2.8198e-09 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 221/300
224/224 [==============================] - 0s 155us/sample - loss: 2.8814e-09 - ac

224/224 [==============================] - 0s 141us/sample - loss: 7.1114e-11 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 269/300
224/224 [==============================] - 0s 162us/sample - loss: 4.9701e-11 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 270/300
224/224 [==============================] - 0s 162us/sample - loss: 4.6783e-11 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 271/300
224/224 [==============================] - 0s 138us/sample - loss: 3.6994e-11 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 272/300
224/224 [==============================] - 0s 142us/sample - loss: 3.6815e-11 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 273/300
224/224 [==============================] - 0s 139us/sample - loss: 2.6026e-11 - accuracy: 1.0000 - val_loss: 0.0034 - val_accuracy: 0.3750
Epoch 274/300
224/224 [==============================] - 0s 168us/sample - loss: 2.1091e-11 - ac

In [ ]:
model.predict()

In [51]:
#得到文档元素对象，拿出<data>间的数据
root = dom.documentElement
cp=dom.getElementsByTagName('position')
c1=cp[0]
c2=c1.firstChild.data
 
#先根据换行符拆分
result=c2.split('\n')
#得到的list第一行为空，去除这行
result.remove(result[0])
 
#根据空格把数字拆分出来，得到每行list，切片将空值切去
b=[]
for lines in result:
    c=lines.split(' ')
    c=c[4:]
    for j in c:
        b.append(j)
 
#转化为mat
b=np.array(b)
#源数据得到424*512，上面数据集没有这么多元素
mat=b.reshape(424,512)
df=pd.Dataframe(mat)

ValueError: cannot reshape array of size 0 into shape (424,512)